REFRESH FUNCTION:

In [ ]:
load_dotenv()


class PowerBIRefreshManager:
    def __init__(self):
        self.client_id = os.getenv('CLIENT_ID')
        self.client_secret = os.getenv('CLIENT_SECRET')
        self.tenant_id = os.getenv('TENANT_ID')
        self.workspace_id = os.getenv('WORKSPACE_ID')
        self.dataset_id = os.getenv('DATASET_ID')
        self.ph_timezone = pytz.timezone('Asia/Manila')
   
    def format_to_ph_time(self, utc_str):
        """Convert UTC string to Philippines time"""
        utc_dt = datetime.strptime(utc_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        utc_dt = pytz.utc.localize(utc_dt)
        ph_dt = utc_dt.astimezone(self.ph_timezone)
        return ph_dt.strftime("%Y-%m-%d %H:%M:%S")


    def get_access_token(self, scope):
        """Get access token for the given scope"""
        authority = f'https://login.microsoftonline.com/{self.tenant_id}'
        app = msal.ConfidentialClientApplication(
            self.client_id,
            authority=authority,
            client_credential=self.client_secret
        )
        result = app.acquire_token_for_client(scopes=[scope])
        if 'access_token' in result:
            return result['access_token']
        else:
            raise Exception(f"Failed to get access token: {result.get('error_description', 'Unknown error')}")


    def get_refresh_history(self):
        """Get the refresh history of the dataset"""
        try:
            access_token = self.get_access_token('https://analysis.windows.net/powerbi/api/.default')
            headers = {
                'Authorization': f'Bearer {access_token}',
                'Content-Type': 'application/json'
            }


            url = f'https://api.powerbi.com/v1.0/myorg/groups/{self.workspace_id}/datasets/{self.dataset_id}/refreshes'
            response = requests.get(url, headers=headers)


            if response.status_code == 200:
                refreshes = response.json().get('value', [])
                formatted_refreshes = []
               
                for refresh in refreshes:
                    # Get user information for this refresh
                    user_info = "System"  # Default value
                    if refresh.get('serviceExecution') == 'User':
                        request_id = refresh.get('requestId')
                        if request_id:
                            # Get user details from the refresh details endpoint
                            details_url = f'https://api.powerbi.com/v1.0/myorg/groups/{self.workspace_id}/datasets/{self.dataset_id}/refreshes/{request_id}'
                            details_response = requests.get(details_url, headers=headers)
                            if details_response.status_code == 200:
                                details = details_response.json()
                                user_info = details.get('refreshedBy', 'Unknown User')


                    formatted_refreshes.append({
                        'status': refresh.get('status'),
                        'timestamp': self.format_to_ph_time(refresh.get('startTime')),
                        'refreshed_by': user_info,
                        'error': refresh.get('error', {}).get('message') if refresh.get('status') == 'Failed' else None
                    })
               
                return formatted_refreshes
            else:
                raise Exception(f"Failed to get refresh history: {response.status_code} - {response.text}")
        except Exception as e:
            raise Exception(f"Error getting refresh history: {str(e)}")


    def refresh_dataset(self):
        """Refresh the dataset"""
        try:
            access_token = self.get_access_token('https://analysis.windows.net/powerbi/api/.default')
            headers = {
                'Authorization': f'Bearer {access_token}',
                'Content-Type': 'application/json'
            }


            url = f'https://api.powerbi.com/v1.0/myorg/groups/{self.workspace_id}/datasets/{self.dataset_id}/refreshes'
            response = requests.post(url, headers=headers)


            if response.status_code != 202:
                raise Exception(f"Failed to refresh dataset: {response.status_code} - {response.text}")


        except Exception as e:
            raise Exception(f"Error refreshing dataset: {str(e)}")


class SharePointFilesManager:
    def __init__(self):
        self.client_id = os.getenv('CLIENT_ID')
        self.client_secret = os.getenv('CLIENT_SECRET')
        self.tenant_id = os.getenv('TENANT_ID')
        self.sharepoint_site_url = os.getenv('SHAREPOINT_SITE_URL')
        self.folder_path = os.getenv('SHAREPOINT_FOLDER_PATH', '').strip('/')  # Remove leading/trailing slashes
        self.excel_file_name = os.getenv('EXCEL_FILE_NAME')
       
        # Validate required environment variables
        if not all([self.client_id, self.client_secret, self.tenant_id, self.sharepoint_site_url]):
            raise Exception("Missing required environment variables. Please check CLIENT_ID, CLIENT_SECRET, TENANT_ID, and SHAREPOINT_SITE_URL")
   
    def get_access_token(self, scope):
        """Get access token for the given scope"""
        authority = f'https://login.microsoftonline.com/{self.tenant_id}'
        app = msal.ConfidentialClientApplication(
            self.client_id,
            authority=authority,
            client_credential=self.client_secret
        )
        result = app.acquire_token_for_client(scopes=[scope])
        if 'access_token' in result:
            return result['access_token']
        else:
            raise Exception(f"Failed to get access token: {result.get('error_description', 'Unknown error')}")


    def get_drive_id(self):
        """Get the Drive ID of the SharePoint document library"""
        print("\nGetting drive ID...")
        access_token = self.get_access_token('https://graph.microsoft.com/.default')
        headers = {
            'Authorization': f'Bearer {access_token}',
            'Accept': 'application/json'
        }
        site_url_parts = self.sharepoint_site_url.split('/sites/')
        hostname = site_url_parts[0].replace('https://', '')
        site_name = site_url_parts[1]


        print(f"Hostname: {hostname}")
        print(f"Site Name: {site_name}")


        url = f"https://graph.microsoft.com/v1.0/sites/{hostname}:/sites/{site_name}:/drives"
        print(f"Requesting drives from: {url}")
       
        response = requests.get(url, headers=headers)


        if response.status_code == 200:
            drives = response.json().get('value', [])
            print("\nAvailable drives in the site:")
            for drive in drives:
                print(f"- Name: {drive.get('name')}")
                print(f"  ID: {drive.get('id')}")
                print(f"  Description: {drive.get('description', 'No description')}")
                print(f"  Drive Type: {drive.get('driveType')}")
                print("---")
           
            # Try to find the documents drive with different possible names
            document_drive_names = ['Shared Documents', 'Documents', 'Document Library', 'Shared Documents Library']
            for drive_name in document_drive_names:
                for drive in drives:
                    if drive.get('name') == drive_name:
                        drive_id = drive.get('id')
                        print(f"\nFound matching drive: {drive_name} with ID: {drive_id}")
                        return drive_id
           
            raise Exception(f"Could not find a documents drive. Available drives: {[drive.get('name') for drive in drives]}")
        else:
            print(f"Error getting drives. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            raise Exception(f"Failed to retrieve drives: {response.status_code} - {response.text}")


    def get_files(self):
        """Get list of files from SharePoint"""
        try:
            print("\nStarting SharePoint file retrieval...")
            print(f"SharePoint Site URL: {self.sharepoint_site_url}")
            print(f"Folder Path: {self.folder_path}")
           
            drive_id = self.get_drive_id()
           
            access_token = self.get_access_token('https://graph.microsoft.com/.default')
            headers = {
                'Authorization': f'Bearer {access_token}',
                'Accept': 'application/json'
            }


            # If folder_path is "Shared Documents", treat it as the root of the Documents drive
            if self.folder_path.lower() in ['shared documents', 'documents']:
                items_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
                print("\nAccessing root of Documents drive")
            else:
                # For any other folder path, use it as a subfolder
                items_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{self.folder_path}:/children"
                print(f"\nAccessing folder path: {self.folder_path}")
           
            print(f"Files URL: {items_url}")
            response = requests.get(items_url, headers=headers)
           
            if response.status_code != 200:
                print(f"\nFiles request failed:")
                print(f"Status Code: {response.status_code}")
                print(f"Response: {response.text}")
                raise Exception(f"Error listing files: {response.status_code} - {response.text}")
           
            items = response.json().get('value', [])
            print(f"\nSuccessfully retrieved {len(items)} items")
           
            # Format the items for display
            formatted_items = [{
                'name': item.get('name'),
                'modified': datetime.strptime(item.get('lastModifiedDateTime'), "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d %H:%M:%S"),
                'url': item.get('webUrl')
            } for item in items if not item.get('folder')]  # Exclude folders
           
            return formatted_items
           
        except Exception as e:
            print(f"\nError in get_files: {str(e)}")
            print("Stack trace:")
            import traceback
            traceback.print_exc()
            raise


if __name__ == '__main__':
    powerbi = PowerBIRefreshManager()
    sharepoint = SharePointFilesManager()
   
    try:
        # Test PowerBI refresh
        print("\nTesting PowerBI refresh...")
        history = powerbi.get_refresh_history()
        print(f"Found {len(history)} refresh records")
       
        # Test SharePoint files
        print("\nTesting SharePoint files...")
        files = sharepoint.get_files()
        print(f"Found {len(files)} files")
       
    except Exception as e:
        print(f"Error during testing: {str(e)}")

APP CONNECT:

In [ ]:

from flask import Flask, render_template, jsonify
from powerbi_refresh import SharePointFilesManager, PowerBIRefreshManager
import os
from dotenv import load_dotenv


load_dotenv()


app = Flask(__name__)


# Initialize managers
sharepoint_manager = SharePointFilesManager()
powerbi_manager = PowerBIRefreshManager()


@app.route('/')
def dashboard():
    return render_template('dashboard.html')


@app.route('/refresh-history')
def refresh_history():
    try:
        history = powerbi_manager.get_refresh_history()
        return render_template('refresh_history.html', history=history)
    except Exception as e:
        return render_template('refresh_history.html', error=str(e))


@app.route('/api/refresh-history')
def api_refresh_history():
    try:
        history = powerbi_manager.get_refresh_history()
        return jsonify({'refreshes': history})
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/sharepoint-files')
def sharepoint_files():
    try:
        files = sharepoint_manager.get_files()
        return render_template('sharepoint_files.html', files=files)
    except Exception as e:
        return render_template('sharepoint_files.html', error=str(e))


@app.route('/refresh-data', methods=['POST'])
def refresh_data():
    try:
        powerbi_manager.refresh_dataset()
        return {'status': 'success', 'message': 'Refresh initiated successfully'}
    except Exception as e:
        return {'status': 'error', 'message': str(e)}, 500


if __name__ == '__main__':
    app.run(debug=True)

BASE WEBSITE:

In [ ]:
# Refresh
<span id="refresh-status" class="text-sm text-gray-500"></span>
<button onclick="refreshData()" class="inline-flex items-center px-4 py-2 bg-blue-600 text-white rounded-md hover:bg-blue-700">
    <svg class="w-4 h-4 mr-2" viewBox="0 0 20 20" fill="currentColor">
        <path fill-rule="evenodd" d="M4 2a1 1 0 011 1v2.101a7.002 7.002 0 0111.601 2.566..." clip-rule="evenodd"/>
    </svg>
    Refresh Data
</button>


# Update Refresh Status
async function updateRefreshStatus() {
    try {
        const response = await fetch('/api/refresh-history');
        const data = await response.json();
        if (data.refreshes && data.refreshes.length > 0) {
            const latestRefresh = data.refreshes[0];
            document.getElementById('refresh-status').textContent =
                `Last refresh: ${formatDate(latestRefresh.timestamp)} - ${latestRefresh.status}`;
        }
    } catch (error) {
        console.error('Error updating refresh status:', error);
    }
}
document.addEventListener('DOMContentLoaded', updateRefreshStatus);



# Refresh Function
function refreshData() {
    const button = document.querySelector('button');
    const statusSpan = document.getElementById('refresh-status');
    button.disabled = true;
    statusSpan.textContent = 'Initiating refresh...';
    fetch('/refresh-data', { method: 'POST' })
        .then(response => response.json())
        .then(data => {
            if (data.status === 'success') {
                statusSpan.textContent = 'Refresh in progress...';
                window.location.href = '/refresh-history';
            } else {
                statusSpan.textContent = 'Failed to initiate refresh';
                button.disabled = false;
            }
        })
        .catch(error => {
            console.error('Error refreshing data:', error);
            statusSpan.textContent = 'Failed to initiate refresh';
            button.disabled = false;
        });
}


DASHBOARD :

In [ ]:
# Dynamic Date Formatting
function formatDate(date) {
    const options = {
        timeZone: 'Asia/Manila',
        year: 'numeric',
        month: '2-digit',
        day: '2-digit',
        hour: '2-digit',
        minute: '2-digit',
        second: '2-digit',
        hour12: false
    };
    return new Date(date).toLocaleString('en-US', options)
        .replace(/(\d+)\/(\d+)\/(\d+),/, '$3-$1-$2')  // Convert MM/DD/YYYY to YYYY-MM-DD
        .replace(/,/, '');  // Remove comma
}

# Fetching and Displaying Latest Refresh Status
async function updateRefreshStatus() {
    try {
        const response = await fetch('/api/refresh-history');
        const data = await response.json();
       
        if (data.error) return console.error('Error:', data.error);

        const refreshes = data.refreshes;
        if (refreshes?.length > 0) {
            const latestRefresh = refreshes[0];
            document.getElementById('refresh-status').textContent =
                `Last refresh: ${formatDate(latestRefresh.timestamp)} - ${latestRefresh.status}`;
        }
    } catch (error) {
        console.error('Error updating refresh status:', error);
    }
}

document.addEventListener('DOMContentLoaded', updateRefreshStatus);


# Refresh Button Action
function refreshData() {
    const button = document.querySelector('button');
    const statusSpan = document.getElementById('refresh-status');
   
    button.disabled = true;
    statusSpan.textContent = 'Initiating refresh...';
   
    fetch('/refresh-data', { method: 'POST' })
    .then(res => res.json())
    .then(data => {
        if (data.status === 'success') {
            statusSpan.textContent = 'Refresh in progress...';
            window.location.href = '/refresh-history';
        } else {
            statusSpan.textContent = 'Failed to initiate refresh';
            button.disabled = false;
        }
    })
    .catch(error => {
        console.error('Error refreshing data:', error);
        statusSpan.textContent = 'Failed to initiate refresh';
        button.disabled = false;
    });
}


HTML FOR SHAREPOINT:

In [ ]:
{% extends "base.html" %}


{% block title %}SharePoint Files / ARISE {% endblock %}


{% block content %}
<div class="space-y-4">
   
    <div class="flex justify-between items-center">
        <div>
            <h1 class="text-2xl font-semibold text-gray-900">SharePoint Files</h1>
            <p class="text-sm text-gray-500 mt-1">Files in the connected SharePoint folder</p>
        </div>
        <button onclick="window.location.reload()" class="inline-flex items-center px-3 py-2 border border-gray-300 shadow-sm text-sm font-medium rounded-md text-gray-700 bg-white hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500">
            <svg class="w-4 h-4 mr-2" viewBox="0 0 20 20" fill="currentColor">
                <path fill-rule="evenodd" d="M4 2a1 1 0 011 1v2.101a7.002 7.002 0 0111.601 2.566 1 1 0 11-1.885.666A5.002 5.002 0 005.999 7H9a1 1 0 010 2H4a1 1 0 01-1-1V3a1 1 0 011-1zm.008 9.057a1 1 0 011.276.61A5.002 5.002 0 0014.001 13H11a1 1 0 110-2h5a1 1 0 011 1v5a1 1 0 11-2 0v-2.101a7.002 7.002 0 01-11.601-2.566 1 1 0 01.61-1.276z" clip-rule="evenodd"/>
            </svg>
            Refresh List
        </button>
    </div>
   
    <div class="bg-white rounded-lg shadow-lg overflow-hidden">
        <div class="p-6">


            {% if error %}
            <div class="bg-red-50 border border-red-200 rounded-lg p-4">
                <div class="flex items-start">
                    <div class="flex-shrink-0">
                        <svg class="h-5 w-5 text-red-400" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 20 20" fill="currentColor">
                            <path fill-rule="evenodd" d="M10 18a8 8 0 100-16 8 8 0 000 16zM8.707 7.293a1 1 0 00-1.414 1.414L8.586 10l-1.293 1.293a1 1 0 101.414 1.414L10 11.414l1.293 1.293a1 1 0 001.414-1.414L11.414 10l1.293-1.293a1 1 0 00-1.414-1.414L10 8.586 8.707 7.293z" clip-rule="evenodd" />
                        </svg>
                    </div>
                    <div class="ml-3">
                        <h3 class="text-sm font-medium text-red-800">Failed to load SharePoint files</h3>
                        <div class="mt-2 text-sm text-red-700">
                            <p>{{ error }}</p>
                        </div>
                        <div class="mt-4">
                            <button onclick="window.location.reload()" class="inline-flex items-center px-3 py-2 border border-red-600 text-sm leading-4 font-medium rounded-md text-red-700 bg-red-50 hover:bg-red-100 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-red-500">
                                Try Again
                            </button>
                        </div>
                    </div>
                </div>
            </div>


            {% else %}
            {% if files %}
            <div class="space-y-2">
                {% for file in files %}
                <div class="file-item flex justify-between items-center p-4 {% if loop.index % 2 == 0 %}bg-gray-50{% endif %} rounded-lg">
                    <div class="flex items-center space-x-3">
                        <svg class="h-5 w-5 text-gray-400" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 20 20" fill="currentColor">
                            <path fill-rule="evenodd" d="M4 4a2 2 0 012-2h4.586A2 2 0 0112 2.586L15.414 6A2 2 0 0116 7.414V16a2 2 0 01-2 2H6a2 2 0 01-2-2V4z" clip-rule="evenodd" />
                        </svg>
                        <div>
                            <p class="font-medium text-gray-900">{{ file.name }}</p>
                            <p class="text-sm text-gray-500">Last modified: {{ file.modified }}</p>
                        </div>
                    </div>
                    <a href="{{ file.url }}" target="_blank" class="inline-flex items-center px-4 py-2 border border-gray-300 rounded-md text-sm font-medium text-gray-700 bg-white hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500">
                        View
                        <svg class="ml-2 -mr-1 h-4 w-4" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 20 20" fill="currentColor">
                            <path d="M11 3a1 1 0 100 2h2.586l-6.293 6.293a1 1 0 101.414 1.414L15 6.414V9a1 1 0 102 0V4a1 1 0 00-1-1h-5z" />
                            <path d="M5 5a2 2 0 00-2 2v8a2 2 0 002 2h8a2 2 0 002-2v-3a1 1 0 10-2 0v3H5V7h3a1 1 0 000-2H5z" />
                        </svg>
                    </a>
                </div>
                {% endfor %}
            </div>
            {% else %}
            <div class="text-center py-12">
                <svg class="mx-auto h-12 w-12 text-gray-400" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                    <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M5 19a2 2 0 01-2-2V7a2 2 0 012-2h4l2 2h4a2 2 0 012 2v1M5 19h14a2 2 0 002-2v-5a2 2 0 00-2-2H9a2 2 0 00-2 2v5a2 2 0 01-2 2z" />
                </svg>
                <h3 class="mt-2 text-sm font-medium text-gray-900">No files found</h3>
                <p class="mt-1 text-sm text-gray-500">No files are currently available in the connected SharePoint folder.</p>
            </div>
            {% endif %}
            {% endif %}
        </div>
    </div>
</div>
{% endblock %}


Python ARIMA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = dataset  # Power BI auto-passes the dataset variable


df['DATE RECEIVED'] = pd.to_datetime(df['DATE RECEIVED'], errors='coerce')


monthly_tickets = df.resample('M', on='DATE RECEIVED').size()


plt.figure(figsize=(10, 6))
plt.plot(monthly_tickets.index, monthly_tickets.values, marker='o', color='b')
plt.xlabel('Month')
plt.ylabel('Ticket Volume')
plt.title('Ticket Volume per Month')

plt.xticks(rotation=45)
plt.tight_layout()


plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA


df = dataset
df['DATE RECEIVED'] = pd.to_datetime(df['DATE RECEIVED'], errors='coerce')
df = df.dropna(subset=['DATE RECEIVED'])


monthly_data = df.resample('M', on='DATE RECEIVED').size()


model = ARIMA(monthly_data, order=(5, 1, 0))
model_fitted = model.fit()


forecast_steps = 6
forecast = model_fitted.forecast(steps=forecast_steps)


last_date = monthly_data.index[-1]
future_dates = pd.date_range(last_date + pd.offsets.MonthBegin(1), periods=forecast_steps, freq='MS')


# Plot actual + forecast
plt.figure(figsize=(12, 6))
plt.plot(monthly_data.index, monthly_data, label='Actual Tickets', color='blue')
plt.plot(future_dates, forecast, label='Forecasted Tickets', color='red', linestyle='--')


plt.title('Monthly Ticket Volume + 6-Month Forecast (ARIMA)')
plt.xlabel('Month')
plt.ylabel('Ticket Volume')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()